In [1]:
#ETL 

In [27]:
import pandas as pd

# from config import API_KEY as key
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import time
import pprint

# from bs4 import BeautifulSoup
import psycopg2
import csv

In [3]:
# pull 2 data sources 

# 3. beautifulSoup 
#review webscrapper.py


In [4]:
#data = list(csv.reader(open(‘data.csv'))) # convert a csv to a 2d list


eth_path = '../data/coin_Ethereum.csv'

eth = pd.read_csv(eth_path)

#load csv
df = pd.DataFrame(eth)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SNo        2031 non-null   int64  
 1   Name       2031 non-null   object 
 2   Symbol     2031 non-null   object 
 3   Date       2031 non-null   object 
 4   High       2031 non-null   float64
 5   Low        2031 non-null   float64
 6   Open       2031 non-null   float64
 7   Close      2031 non-null   float64
 8   Volume     2031 non-null   float64
 9   Marketcap  2031 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 158.8+ KB


In [5]:
df

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Ethereum,ETH,2015-08-08 23:59:59,2.798810,0.714725,2.793760,0.753325,6.741880e+05,4.548689e+07
1,2,Ethereum,ETH,2015-08-09 23:59:59,0.879810,0.629191,0.706136,0.701897,5.321700e+05,4.239957e+07
2,3,Ethereum,ETH,2015-08-10 23:59:59,0.729854,0.636546,0.713989,0.708448,4.052830e+05,4.281836e+07
3,4,Ethereum,ETH,2015-08-11 23:59:59,1.131410,0.663235,0.708087,1.067860,1.463100e+06,6.456929e+07
4,5,Ethereum,ETH,2015-08-12 23:59:59,1.289940,0.883608,1.058750,1.217440,2.150620e+06,7.364501e+07
...,...,...,...,...,...,...,...,...,...,...
2026,2027,Ethereum,ETH,2021-02-23 23:59:59,1781.409018,1378.840907,1781.409018,1570.203925,5.202986e+10,1.802510e+11
2027,2028,Ethereum,ETH,2021-02-24 23:59:59,1710.983795,1511.018963,1571.476497,1626.575705,3.132900e+10,1.867440e+11
2028,2029,Ethereum,ETH,2021-02-25 23:59:59,1670.224062,1465.058946,1625.393882,1475.703735,2.448168e+10,1.694427e+11
2029,2030,Ethereum,ETH,2021-02-26 23:59:59,1559.028891,1407.979253,1478.653270,1446.033650,3.143600e+10,1.660555e+11


## api

In [38]:
#api

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'100',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '1c07eee4-35f6-405c-8c43-aeb0b8682976',
#   'X-CMC_PRO_API_KEY': key,
}



session = Session()
session.headers.update(headers)

data = {}

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
#   print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

#remove status dict 
cleaned_data = data['data']
# print(cleaned_data)

# def subset(headers):
#     #list of return rows 
#     results = []
    
#     for row in cleaned_data:
#         query = []
#         # loop through keys for each row, find interested headers
#         # keys is a built in method for dict
#         for key in row.keys():
#             if key in headers:
#                 #cast value to str 
#                 query.append(str(row[key]))
#         #format results by adding ',' and convert to str
#         formatted_row = ','.join(query)
        
#         results.append(formatted_row)
#     return results
        

def subset(headers):
    #list of return rows 
    results = []
    
    #check if header values is nested or not 
    nested_header = []
    
    #remember headers that cant be found in main query, then must be nested 
    for header in headers:
        #use .keys method on dict, returns all headers 
        if header not in cleaned_data[0].keys():
            nested_header.append(header)
            
    
    for row in cleaned_data:
        query = []
        # loop through keys for each row, find interested headers
        # keys is a built in method for dict
        for key in row.keys():
            if key in headers:
                #cast value to str 
                query.append(str(row[key]))
#             else:
#                 for nested_header in row['quote']['USD'].keys():
#                     if nested_header in headers:
#                         query.append(str(row['quote']['USD'][nested_header]))
        
        for header in nested_header:
            if header in row['quote']['USD'].keys():
                query.append(str(int(row['quote']['USD'][header])))       
        #format results by adding ',' and convert to str
#         formatted_row = ','.join(query)
#         results.append(formatted_row)
        results.append(query)
    return results
        

                
# data_eth = []

# for row in cleaned_data:
# #     print(row['id'] + ' : ' + row['slug'])
# #     print(f"{row['id']} : {row['slug']}")
    
#     if 'ethereum' in row['slug']:
#         data_eth.append(f"{row['id']} : {row['slug']}")
        
# for row in data_eth:
#     print(row)


get_data = subset(['id','symbol','name','price','last_updated'])

for row in get_data:
    print(row)




['1', 'Bitcoin', 'BTC', '2021-06-12T02:29:02.000Z', '35743']
['1027', 'Ethereum', 'ETH', '2021-06-12T02:29:02.000Z', '2268']
['825', 'Tether', 'USDT', '2021-06-12T02:28:09.000Z', '1']
['1839', 'Binance Coin', 'BNB', '2021-06-12T02:28:09.000Z', '329']
['2010', 'Cardano', 'ADA', '2021-06-12T02:28:12.000Z', '1']
['74', 'Dogecoin', 'DOGE', '2021-06-12T02:29:03.000Z', '0']
['52', 'XRP', 'XRP', '2021-06-12T02:29:03.000Z', '0']
['3408', 'USD Coin', 'USDC', '2021-06-12T02:29:05.000Z', '1']
['6636', 'Polkadot', 'DOT', '2021-06-12T02:29:06.000Z', '20']
['7083', 'Uniswap', 'UNI', '2021-06-12T02:28:10.000Z', '20']
['1831', 'Bitcoin Cash', 'BCH', '2021-06-12T02:29:08.000Z', '552']
['2', 'Litecoin', 'LTC', '2021-06-12T02:29:02.000Z', '155']
['4687', 'Binance USD', 'BUSD', '2021-06-12T02:29:07.000Z', '1']
['5426', 'Solana', 'SOL', '2021-06-12T02:29:05.000Z', '34']
['1975', 'Chainlink', 'LINK', '2021-06-12T02:28:10.000Z', '20']
['2416', 'THETA', 'THETA', '2021-06-12T02:29:05.000Z', '7']
['3890', 'Poly

In [40]:
#load 

conn = psycopg2.connect("host=localhost dbname=etl")
cur = conn.cursor()

for row in get_data:
#     print("INSERT INTO data VALUES (" + str(row[0]) + "," + "'" + row[1] + "'" + "," + "'" + row[2] + "'" + "," + "'" +row[3] + "'" + "," + str(row[4]) + ")")
    cur.execute("INSERT INTO data VALUES (" + str(row[0]) + "," + "'" + row[1] + "'" + "," + "'" + row[2] + "'" + "," + "'" +row[3] + "'" + "," + str(row[4]) + ")")

    
    
# with open('user_accounts.csv', 'r') as f:
#     reader = csv.reader(f)
#     next(reader) # Skip the header row.
#     for row in reader:
#         cur.execute(
#         "INSERT INTO users VALUES (%s, %s, %s, %s)",
#         row
#     )
# conn.commit()




# cur.execute("INSERT INTO data VALUES(2,'ETHEREUM','ETC')")
cur.execute('SELECT * FROM data')

all = cur.fetchall()
conn.commit()

print(all)

[(1, 'Bitcoin', 'BTC', '2021-06-12T02:29:02.000Z', 35743), (1027, 'Ethereum', 'ETH', '2021-06-12T02:29:02.000Z', 2268), (825, 'Tether', 'USDT', '2021-06-12T02:28:09.000Z', 1), (1839, 'Binance Coin', 'BNB', '2021-06-12T02:28:09.000Z', 329), (2010, 'Cardano', 'ADA', '2021-06-12T02:28:12.000Z', 1), (74, 'Dogecoin', 'DOGE', '2021-06-12T02:29:03.000Z', 0), (52, 'XRP', 'XRP', '2021-06-12T02:29:03.000Z', 0), (3408, 'USD Coin', 'USDC', '2021-06-12T02:29:05.000Z', 1), (6636, 'Polkadot', 'DOT', '2021-06-12T02:29:06.000Z', 20), (7083, 'Uniswap', 'UNI', '2021-06-12T02:28:10.000Z', 20), (1831, 'Bitcoin Cash', 'BCH', '2021-06-12T02:29:08.000Z', 552), (2, 'Litecoin', 'LTC', '2021-06-12T02:29:02.000Z', 155), (4687, 'Binance USD', 'BUSD', '2021-06-12T02:29:07.000Z', 1), (5426, 'Solana', 'SOL', '2021-06-12T02:29:05.000Z', 34), (1975, 'Chainlink', 'LINK', '2021-06-12T02:28:10.000Z', 20), (2416, 'THETA', 'THETA', '2021-06-12T02:29:05.000Z', 7), (3890, 'Polygon', 'MATIC', '2021-06-12T02:28:09.000Z', 1), (5

In [8]:
# transform
# perform data operation 
# dedupe, remove outliers 

In [9]:
# Load 

#SQL, postgres 